Generating Raw Data Workshop
==========================

The `litl_simulator` python package does 2 things. First it generates raw data using the CARLA simulation environment. Secondly, it processes this raw data into waveforms which it then feeds to a simulated DSP that transforms them into a 3D point cloud + intensities. This notebook is dedicated to the first part which also explains how to setup CARLA.


Setup CARLA
--------------------

We are using a customized version of CARLA in order to be able to return a fixed size raw point cloud array. Also, there is a customized shader within it that is used to compute realistic point cloud intensities. Everything has been compiled into a docker image (see README for download link).
 
After a decade of download, you should be able to run the image straightforwardly. But first (on ubuntu), you need to allow the docker container to access the xhost. Then, you can boot the docker container which should display the default city (see image below).

```console
$ xhost local:root 
$ docker run --user ue4 --privileged --gpus all --net=host -v /tmp/.X11-unix:/tmp/.X11-unix:rw -e DISPLAY=:1 -e SDL_VIDEODRIVER=x11 carla_lidar_sim /bin/bash /home/ue4/carla/Dist/CARLA_Shipping_9071d25-dirty/LinuxNoEditor/CarlaUE4.sh
```

<img src="workshops_images/boot_carla.png" alt="What should appear when booting up CARLA image">

Note here that the `--user ue4` option is mandatory since the unreal engine (the underlying engine ontop of which CARLA is built cannot run as root).

From that point, it is possible to move around the world by using the ASDW keys and the mouse. Briefly, the CARLA engine works as a server which accepts commands from a specialized CARLA client. Since this workshop is not a tutorial on CARLA, please refer to [the CARLA documention](https://carla.readthedocs.io/en/latest/) in order to use this server. The only things different with the vanilla version of CARLA is the `semantic LiDAR` and the new camera blueprints called `diffuse_color` and `specular_metallic_roughness` which work the same way as the other cameras.

The last thing to do is install the CARLA client python library specially built for this version of CARLA. It is stored at the top of this gitlab repo (`carla-0.9.13-cp38-cp38-linux_x86_64.whl`). Just install it using:

```console
$ pip install carla-0.9.13-cp38-cp38-linux_x86_64.whl
```
Note, this python library only works for `python 3.8`. At any point, the only way to kill the CARLA server is by stopping/removing the container using `docker`.

Generate Raw Data
----------------------------

The `lidar_carla_sim` python package has been developed to generate raw data using the CARLA client API. After `pip install`ing it and running the CARLA server in a docker container, it is possible to start simulating raw data.

In [6]:
import os

from litl_simulator import LiDARSim

In [1]:
# There are multiple maps in CARLA. In order to use the other ones, uncomment the following lines.
# Changing worlds need to be done before using the LiDARSim object.
# import carla
# client = carla.Client('localhost', 2000)  # create carla client
# worlds = [world for world in client.get_available_maps() if 'Opt' in world]  # list of available worlds
# world = worlds[5]  # choose world
# client.load_world(world)  # load world

# create simulation object
sim = LiDARSim(
    fixed_delta_seconds=0.1,  # time resolution for each world tick (seconds)
    # loglevel=logging.DEBUG,  # logging option use this to see more logs
    timeout=100,  # (seconds) timeout before client decides to throw error. Use this in case your computer is super slow
)

INFO:LiDARSim:Synchro = True, fixed delta seconds = 0.1, TM synchronous
INFO:ActorManager:Destroying all already existing actors.


In [2]:
# NOTE: after creating the LiDARSim object, the CARLA server will be set in "synchronous" mode where each
# world tick is controlled by the client object. Thus, it will appear as the server is frozen but it has
# not crashed! It is just paused by the client actually. In order to still travel the world, you need to call
# the run method.

sim.run()  # use the stop button in jupyter notebook to continue (this is an underlying while True method.)

INFO:LiDARSim:Running simulation...
INFO:LiDARSim:Simulation stopped. Total simulation time: 17.9s.


In [2]:
# spawn ego vehicle and NPC vehicles at random locations
sim.spawn_npc_vehicles(
    max_n_vehicles=60,  # max number of vehicles. Note: if too high CARLA might not be able to spawn them all
    ego=True,  # spawn or not the ego vehicle. If set to False it will be impossible to spawn sensors.
)
# spawn pedestrians
sim.spawn_npc_pedestrians(
    max_n_pedestrians=50,  # same meaning as max_n_vehicles
    percentage_pedestrians_running=10,  # self explanatory
    percentage_pedestrians_crossing=50,  # % chance that a pedestrian decides to cross the street
)

INFO:ActorManager:Spawning maximum 60 vehicles.
INFO:ActorManager:Spawning first 1 regular vehicle.
INFO:ActorManager:Spawning maximum 59 vehicles of any type.
INFO:ActorManager:Managed to spawn a total of 60 vehicles!
INFO:ActorManager:Setting ego vehicle...
INFO:ActorManager:The ego vehicle is a 'mercedes coupe 2020'.
INFO:ActorManager:Setting pedestrian_cross_factor to 50%
INFO:ActorManager:Managed to spawn n=50 pedestrians! With 10% chance running.


In [ ]:
# NOTE 1: as mentioned, it is possible that the number of spawned actors is less than what is asked.
# In order to spawn more you might need to let the engine run for a while and then retry to spawn more.
# This is because there is a limited number of spawn points and depending on the actors, if there is
# an overlap, spawning will fail.
#
# NOTE 2: Upon spawn, the Traffic Manager from CARLA make the cars move at random destination. If there are
# too many cars, traffic might happen and sometimes everything can be blocked indefinitely.
# If that happens, you might want to restart the simulation (just restart the notebook kernels and rerun cells)
# When you initialize a LiDARSim object, all actors are destroyed.
#
# NOTE 3: Pedestrians are also initialized with an AI so that they move at random locations.
#
# NOTE 4: Vehicles are spawned slightly in the air as to prevent weird collisions with the ground.
# It is thus necessary to run the simulation for a while after spawning vehicles in order to give them time
# to drop.

# To see the world being alive: just call the sim.run() function again. You'll be able to move around then.
sim.run()  # use 'stop' button from jupyter to stop simulation

In [3]:
# To follow an actor while the engine is running, use the 'watch' argument.
sim.run(
    watch=sim.ego_vehicle,  # to follow ego vehicle
    # watch=sim.actor_manager.pedestrians[0],  # to follow a pedestrian
    # watch=sim.actor_manager.vehicles[0],  # to follow a NPC vehicle
)

INFO:LiDARSim:Running simulation...
INFO:LiDARSim:Simulation stopped. Total simulation time: 2.2s.


In [4]:
# Now that you're satisfied with the simulation setup, you can now spawn the sensors.
# NOTE 1: sensors spawn on top of the ego vehicle's position (use z argument to change the height).
# NOTE 2: sensors are invisible; you won't see them in the simulation environment.

upsampling = 5  # LiDAR upsampling ratio. This multiplies the number of channels and pts / channel
                # Remember this setting for later as it will be used to simulate multiple echo effects
                # by downsampling multiple adjacent points into the same wavefront.

sim.spawn_lidar(
    z=2.4,  # height of sensors (not just LiDAR) above GROUND (in meters)
    range_=120,  # maximum LiDAR range (in meters)
    lower_fov=-10, upper_fov=10,  # Vertical FOV in degrees
    rotation_frequency=10,        # Number of rotation in Hz. Preferably this should be 1 / fixed_delta_seconds
                                  # Such that each physical tick contains a whole rotation (since data is
                                  # saved on disk for each frame). Otherwise data should be merged together 
                                  # prior to simulation pipeline.
    horizontal_fov=360,  # Horizontal FOV
    points_per_rotation_per_laser=875 * upsampling,  # number of raw points per channel per rotation
    channels=128 * upsampling,  # number of raw channel
    )

# The following will spawn all the cameras at the same location as the LiDAR
# There will be 4 cameras for each type of camera.
# Each camera for a given type is rotated by 90 degrees from each other such that they see all around.
sim.spawn_cameras(1280, 800)  # Image size in pixels (width, height).

INFO:ActorManager:Spawning LiDAR.
INFO:ActorManager:LiDAR points/rot/laser set to 4375 gives 28000000 total pts/s.
INFO:ActorManager:Setting LiDAR's range = 120.
INFO:ActorManager:Setting LiDAR's channels = 640.
INFO:ActorManager:Setting LiDAR's lower_fov = -10.
INFO:ActorManager:Setting LiDAR's rotation_frequency = 10.
INFO:ActorManager:Setting LiDAR's upper_fov = 10.
INFO:ActorManager:Setting LiDAR's horizontal_fov = 360.
INFO:ActorManager:Setting LiDAR's points_per_second = 28000000.
INFO:ActorManager:Spawning Ego vehicle's cameras.


In [8]:
# Now is the time to stop procrastinating and generate the RAW data for simulation
# NOTE: setting the 'save_sensors_to' to something else than None will make the sensors save data to disk.
# This is super slow!!! So make sure the simulation is set as you'd like BEFORE using this.

storage_dir = os.path.expanduser("~/Workspace/carla_workshop_data")
sim.run(watch=sim.ego_vehicle,  # optional at this point
        # max_frames=500,  # maximum number of frames to save on disk
        # save_frame_over_distance=1,  # use this so that only frames where the ego has traveled at least
                                       # the given distance (in meters) are stored on disk.
                                       # NOTE: this does not mean data is not generated. It is just discarded
                                       # and thus the simulation only goes slightly faster whe
        # save_every_tick=10,  # save frames at every ticks. As for the 'save_frame_over_distance' arg,
                               # sensors actually grab the data at every tick but do not save them on disk
                               # it is thus only a slight gain in speed (default is 1)
        # max_frames=10, # max number of frames to grabn
        max_ticks=10,    # maximum number of world ticks
        save_sensors_to=storage_dir,  # directory where data is stored
       )

INFO:LiDARSim:Running simulation...
INFO:LiDARSim:Simulation progress: 10 %
INFO:LiDARSim:Simulation progress: 20 %
INFO:LiDARSim:Simulation progress: 30 %
INFO:LiDARSim:Simulation progress: 40 %
INFO:LiDARSim:Simulation progress: 50 %
INFO:LiDARSim:Simulation progress: 60 %
INFO:LiDARSim:Simulation progress: 70 %
INFO:LiDARSim:Simulation progress: 80 %
INFO:LiDARSim:Simulation progress: 90 %
INFO:LiDARSim:Simulation progress: 100 %
INFO:LiDARSim:Simulation stopped. Total simulation time: 1.0m1.8s.


**Warning:**
------------------

Unfortunately, if you use either `save_frame_over_distance` or the `save_every_tick` arguments,
the pedestrian bounding boxes will be misplaced sometimes. This is a bug and I am not sure where
it comes from. If pedestrian bounding boxes do not matter to you, then you can use them without
any problem. Otherwise, the trick is to save frames at every ticks and to delete the ones you
don't care. This is slower and use more diskspace momentarily :/